In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import random

# Funciones

In [3]:
df_relojes = pd.DataFrame()

def parsear_relojes(links):
    global df_relojes  # Acceder al DataFrame global
    for link in links:
        # URL del reloj en Sothebys
        url = f"https://www.sothebys.com{link}"
        browser = webdriver.Edge()

        try:
            browser.get(url)
            time.sleep(random.randint(5, 10)) 
            wait = WebDriverWait(browser, 10)

            try:
                wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[2]/div/div[1]/div/div[2]/div/button[2]"))).click()
            except:
                pass
            
            wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div/div[4]/div[2]/div[2]")))

            # Obtener el HTML de la pagina
            html = browser.page_source
            soup = BeautifulSoup(html, "html.parser")
            
            resultados = {}

            marca = soup.find("h1", {"class":"headline-module_headline32Regular__pMgbY"}).text.strip()
            precios = soup.find("div", {"class":"information_pdp_priceRow__wwkqL"})
            if precios:
                p = precios.find_all("p")
                if len(p) > 1:
                    precio = p[1].text.strip()
                else: pass
            else: pass
            resultados["Marca"] = marca
            resultados["Precio"] = precio

            tabla = soup.find("div", {"class":"accordion_body__hE7ic"})
            if tabla:
                filas = tabla.find_all("div")
                for fila in filas[2:-1]:
                    celdas = fila.find_all("p")
                    dato = fila.find_all("span")

                    if celdas and dato:
                    # Guardar los datos en el dic
                        clave = celdas[0].text.strip()
                        valor = dato[0].text.strip()
                        resultados[clave] = valor

                # Agregar los datos extraídos al DataFrame
                df_relojes = pd.concat([df_relojes, pd.DataFrame([resultados])], ignore_index=True)

                # Guardar los datos en un archivo JSON
                with open("datos_relojes_sothebys.json", "w", encoding="utf-8") as archivo_json:
                    df_relojes.to_json(archivo_json, orient="records", force_ascii=False, indent=4)

                # Mostrar los datos extraídos
                print("\n Datos extraídos del reloj:")
                for clave, valor in resultados.items():
                    print(f"{clave}: {valor}")

        except Exception as e:
            print(f"Error general al procesar el reloj {link}: {e}")

        finally:
            # Cerrar el navegador
            browser.quit()

# Data de un Reloj

In [41]:
# URL del reloj en Sothebys
url = "https://www.sothebys.com/en/buy/pdp/luxury/watches/watch/_rolex-a-limited-white-gold-daytona-lemans-with-box-and-5390"
browser = webdriver.Edge()

try:
    browser.get(url)
    wait = WebDriverWait(browser, 10)

    try:
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[2]/div/div[1]/div/div[2]/div/button[2]"))).click()
    except:
        pass
    
    wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div/div[4]/div[2]/div[2]")))

    # Obtener el HTML de la pagina
    html = browser.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    resultados = {}

    marca = soup.find("h1", {"class":"headline-module_headline32Regular__pMgbY"}).text.strip()
    precios = soup.find("div", {"class":"information_pdp_priceRow__wwkqL"})
    if precios:
        p = precios.find_all("p")
        if len(p) > 1:
            precio = p[1].text.strip()
        else: pass
    else: pass
    resultados["Marca"] = marca
    resultados["Precio"] = precio

    tabla = soup.find("div", {"class":"accordion_body__hE7ic"})
    if tabla:
        filas = tabla.find_all("div")
        for fila in filas[2:-1]:
            celdas = fila.find_all("p")
            dato = fila.find_all("span")

            if celdas and dato:
            # Guardar los datos en el dic
                clave = celdas[0].text.strip()
                valor = dato[0].text.strip()
                resultados[clave] = valor

except Exception as e:
    print(F"Error general: {e}")

In [42]:
resultados

{'Marca': 'Rolex',
 'Precio': '264.700 USD',
 'Condition Report': "ReviveFairGoodVery GoodLike NewThe watch is new. The movement is running at the time of cataloging, however, it was not tested for the accuracy of time or duration of the power reserve and may need service at the buyer's discretion. Please note that Sotheby's does not guarantee the future working of the movement.Please be advised that bands made of materials derived from endangered or otherwise protected species (i.e. alligator and crocodile) cannot be shipped outside the country of sale. We reserve the right to remove these bands prior to shipping.Please note that this item is final sale and not eligible for return.",
 'Year': '2024',
 'Reference Number': '126529LN',
 'Model': 'Daytona',
 'Complication': 'Chronograph',
 'Gender': "Men's",
 'Dial Color(s)': 'Black',
 'Caliber': 'Cal. 4132',
 'Movement Type': 'Automatic',
 'Case Material(s)': 'White Gold',
 'Case Size (mm)': '40 mm diameter',
 'Case Back': 'Sapphire disp

# Extraer Links

In [5]:
links = []
marcas = ["rolex", "patek-philippe", "audemars-piguet"]

# URL de toda la pagina
for marca in marcas:
    url = f"https://www.sothebys.com/en/buy/luxury/watches/watch/{marca}"
    browser = webdriver.Edge()

    try:
        browser.get(url)
        time.sleep(random.randint(10,15))
        wait = WebDriverWait(browser, 10)

        try:
            wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[2]/div/div[1]/div/div[2]/div/button[2]"))).click()
        except Exception:
            pass

        wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[5]/div[2]/div")))

        # Obtener el html de la pagina
        html = browser.page_source
        soup = BeautifulSoup(html, "html.parser")

        tabla = soup.find("ul", {"class":"gridContent BrowsePage_hitsContainer__sEAG3"})
        if tabla:
            anuncio = tabla.find_all("a", href = True)
            for reloj in anuncio:
                link = reloj.get("href")
                if link:
                    links.append(link)
        else: pass

    except Exception as e:
        print(f"Error general: {e}")

### Links de las tres marcas

In [6]:
links

['/en/buy/pdp/luxury/watches/watch/_rolex-a-limited-white-gold-daytona-lemans-with-box-and-5390',
 '/en/buy/pdp/luxury/watches/watch/_rolex-cosmograph-daytona-reference-116509-a-white-gold-automatic-wristwatch-with-chronograph-2020-6be4',
 '/en/buy/pdp/luxury/watches/watch/_rolex-sea-dweller-reference-136660-a-stainless-steel-automatic-wristwatch-with-date-2023-ad2b',
 '/en/buy/pdp/luxury/watches/watch/_rolex-datejust-reference-116189-a-white-gold-diamond-set-automatic-wristwatch-with-date-2008-2009-26d6',
 '/en/buy/pdp/luxury/watches/watch/_rolex-cosmograph-daytona-reference-116519-a-white-gold-automatic-wristwatch-with-chronograph-2019-4953',
 '/en/buy/pdp/luxury/watches/watch/_rolex-cosmograph-daytona-reference-116518-a-yellow-gold-diamond-set-automatic-wristwatch-with-chronograph-2018-15cd',
 '/en/buy/pdp/luxury/watches/watch/_rolex-cosmograph-daytona-reference-116520-a-stainless-steel-automatic-wristwatch-with-chronograph-2008-2009-ef76',
 '/en/buy/pdp/luxury/watches/watch/_rolex-

### Mezclar los links 

In [9]:
shuffled_links = links.copy()
random.shuffle(shuffled_links)


shuffled_links = shuffled_links[0:50]
shuffled_links

['/en/buy/pdp/luxury/watches/watch/_millenary-escape-deadbeat-seconds-watch-26091orood803cr01-a61c',
 '/en/buy/pdp/luxury/watches/watch/_audemars-piguet-royal-oak-reference-25860st001110st01-a-stainless-steel-automatic-wristwatch-with-date-and-chronograph-1998-868c',
 '/en/buy/pdp/luxury/watches/watch/_audemars-piguet-remaster-01-limited-edition-reference-26595sr-a-rose-gold-and-stainless-steel-unworn-automatic-wristwatch-with-chronograph-02aa',
 '/en/buy/pdp/luxury/watches/watch/_patek-philippe-calatrava-hunter-case-reference-5227-a-yellow-gold-automatic-wristwatch-with-date-2013-present-23bd',
 '/en/buy/pdp/luxury/watches/watch/_patek-philippe-calatrava-reference-5153j-001-a-yellow-gold-automatic-wristwatch-with-date-2020-816e',
 '/en/buy/pdp/luxury/watches/watch/_rolex-president-day-date-reference-118138-a-yellow-gold-automatic-wristwatch-with-day-and-date-f533',
 '/en/buy/pdp/luxury/watches/watch/_rolex.-day-date-(reference-18948)-a-tri-colour-gold-and-diamond-set-meteorite-dial-wr

In [10]:
parsear_relojes(shuffled_links)


 Datos extraídos del reloj:
Marca: Audemars Piguet
Precio: 85.560 USD
Condition Report: ReviveFairGoodVery GoodLike NewThe watch is in overall very good condition with minor wear throughout commensurate with age. The movement is running at the time of cataloging, however, it was not tested for the accuracy of time or duration of the power reserve and may need service at the buyer's discretion. Please note that Sotheby's does not guarantee the future working of the movement.Please be advised that bands made of materials derived from endangered or otherwise protected species (i.e. alligator and crocodile) cannot be shipped outside the country of sale. We reserve the right to remove these bands prior to shipping.
Reference Number: 26091OR.OO.D803CR.01
Model: Millenary
Gender: Men's
Dial Color(s): Skeletonized
Dial Numerals: Roman
Movement Type: Manual Wind
Case Material(s): Rose Gold
Case Size (mm): 47
Case Back: Sapphire display-back
Band Material: Alligator Skin
Clasp Type: Deployant b

In [11]:
ruta_json = "datos_relojes_sothebys.json"
df_relojes_sothebys = pd.read_json(ruta_json)
df_relojes_sothebys

,Marca,Precio,Condition Report,Reference Number,Model,Gender,Dial Color(s),Dial Numerals,Movement Type,Case Material(s),...,Signature,Papers,SKU,Year,Complication,Dimension Notes,Box,Dial Notes,Caliber,Case Number
0,Audemars Piguet,85.560 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,26091OR.OO.D803CR.01,Millenary,Men's,Skeletonized,Roman,Manual Wind,Rose Gold,...,Dial,Yes,CF6BZ,NaN,None,None,None,None,None,None
1,Audemars Piguet,34.600 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,25860ST.00.1110ST.01,Royal Oak,Men's,Black,Stick,Automatic,Stainless Steel,...,Dial,Yes,D4RCC,1998.0,"Date, Chronograph",Fits a wrist size of 7.5 inches or smaller.,Yes,None,None,None
2,Audemars Piguet,33.900 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,26595SR,None,Men's,None,None,Automatic,Stainless Steel,...,"Dial, Movement",Yes,D2W2B,NaN,Chronograph,None,Yes,None,None,None
3,Patek Philippe,30.500 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,5227,Calatrava,Men's,Beige/Tan,Stick,Automatic,Yellow Gold,...,Dial,None,D2N4P,NaN,Date,None,None,None,None,None
4,Patek Philippe,27.600 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,5153J-001,Calatrava,Men's,Silver,Stick,Automatic,Yellow Gold,...,Dial,Yes,D2MM3,2020.0,Date,Fits a wrist size of 7 inches or smaller.,None,None,None,None
5,Rolex,25.400 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,118138,Day-Date,Men's,Green,Stick,Automatic,Yellow Gold,...,Dial,None,D4ZMH,NaN,"Date, Day of the Week",None,None,None,None,None
6,Rolex,73.200 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,Reference 18948,Day-Date,Men's,None,Diamond-set,Automatic,"Yellow Gold, Rose Gold, White Gold",...,"Case, Dial, Movement",Yes,D6V2X,2015.0,"Day of the Week, Date",Fits a wrist size of 175 mm or smaller.,Yes,Meteorite,Cal. 3155,None
7,Audemars Piguet,58.100 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,26062PT.OO.A002CA.01,Royal Oak Offshore,Men's,Black,Arabic,Automatic,Platinum,...,"Dial, Case",Yes,D2H98,NaN,Chronograph,None,Yes,None,None,None
8,Audemars Piguet,35.600 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,26120OR,Royal Oak,Men's,Black,Stick,Automatic,Rose Gold,...,Dial,None,D2HFM,NaN,"Date, Dual Time, Power Reserve",None,Yes,None,None,None
9,Rolex,264.700 USD,ReviveFairGoodVery GoodLike NewThe watch is ne...,126529LN,Daytona,Men's,Black,None,Automatic,White Gold,...,"Dial, Movement",Yes,D6852,2024.0,Chronograph,None,Yes,None,Cal. 4132,None


In [14]:
df = df_relojes_sothebys[df_relojes_sothebys["Marca"] == "Audemars Piguet"]
df

,Marca,Precio,Condition Report,Reference Number,Model,Gender,Dial Color(s),Dial Numerals,Movement Type,Case Material(s),...,Signature,Papers,SKU,Year,Complication,Dimension Notes,Box,Dial Notes,Caliber,Case Number
0,Audemars Piguet,85.560 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,26091OR.OO.D803CR.01,Millenary,Men's,Skeletonized,Roman,Manual Wind,Rose Gold,...,Dial,Yes,CF6BZ,NaN,None,None,None,None,None,None
1,Audemars Piguet,34.600 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,25860ST.00.1110ST.01,Royal Oak,Men's,Black,Stick,Automatic,Stainless Steel,...,Dial,Yes,D4RCC,1998.0,"Date, Chronograph",Fits a wrist size of 7.5 inches or smaller.,Yes,None,None,None
2,Audemars Piguet,33.900 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,26595SR,None,Men's,None,None,Automatic,Stainless Steel,...,"Dial, Movement",Yes,D2W2B,NaN,Chronograph,None,Yes,None,None,None
7,Audemars Piguet,58.100 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,26062PT.OO.A002CA.01,Royal Oak Offshore,Men's,Black,Arabic,Automatic,Platinum,...,"Dial, Case",Yes,D2H98,NaN,Chronograph,None,Yes,None,None,None
8,Audemars Piguet,35.600 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,26120OR,Royal Oak,Men's,Black,Stick,Automatic,Rose Gold,...,Dial,None,D2HFM,NaN,"Date, Dual Time, Power Reserve",None,Yes,None,None,None
13,Audemars Piguet,69.300 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,26331OR.OO.D821CR.01,Royal Oak,Men's,Brown,Stick,Automatic,Rose Gold,...,"Dial, Case",Yes,D2H2C,NaN,"Date, Chronograph",None,Yes,None,None,None
15,Audemars Piguet,47.000 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,15000TR,Royal Oak,Men's,Pink/Rose,Stick,Automatic,Tantalum,...,Dial,None,D2VCC,NaN,Date,Fits a wrist size of 7 inches or smaller.,None,None,None,None
16,Audemars Piguet,46.850 USD,ReviveFairGoodVery GoodLike New,26400SO.OO.A054CA.01,Royal Oak Offshore,None,Brown,Stick,Automatic,None,...,None,Yes,CHMZ3,NaN,Chronograph,None,Yes,None,3126/3840,K339...
19,Audemars Piguet,70.300 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,26470OR.OO.1000OR.01,Royal Oak Offshore,Men's,"Pink/Rose, Black",Arabic,Automatic,Rose Gold,...,"Dial, Movement",None,D2H28,NaN,"Date, Chronograph",Fits a wrist size of 7.75 inches or smaller.,Yes,None,None,None
28,Audemars Piguet,44.900 USD,ReviveFairGoodVery GoodLike NewThe watch is in...,25730BA,Royal Oak,Unisex,Silver,Stick,Automatic,Yellow Gold,...,Dial,Yes,D2HDB,NaN,"Date, Dual Time, Power Reserve",Fits a wrist size of 7.5 inches or smaller.,Yes,None,None,None
